In [ ]:
import os
import sys
current_dir = os.path.dirname(os.path.abspath('.'))
sys.path.append(current_dir)
os.chdir(current_dir)

In [ ]:
file = "./log/times095.json"

In [ ]:
import json
import re
from pathlib import Path

# 可选的美化库
try:
    from tabulate import tabulate
except ImportError:
    tabulate = None
try:
    from colorama import init as colorama_init, Fore, Style
    colorama_init(autoreset=True)
except ImportError:
    Fore = Style = type("X", (), {"RESET_ALL": "", "BRIGHT": ""})

FILE = Path(file)

pattern = re.compile(
    r'^(commerce|community|trading)__0\.95_4_2_(manual_erase|entity_agnostic|original_query)_evaluate_times$'
)

domains = ["commerce", "community", "trading"]
eval_types = ["manual_erase", "entity_agnostic", "original_query"]

with open(FILE, encoding="utf-8") as f:
    root = json.load(f)
data = root.get("data", {})

# 提取 p90
matrix = {d: {e: None for e in eval_types} for d in domains}
for key, stats in data.items():
    m = pattern.match(key)
    if not m:
        continue
    d, e = m.groups()
    matrix[d][e] = stats.get("p90")
    if e=="manual_erase":
        matrix[d][e] +=(3+0.133)

# 生成二维表数据
table = []
for d in domains:
    row = [d] + [matrix[d][e] for e in eval_types]
    table.append(row)

headers = ["domain"] + eval_types


def format_val(v):
    return f"{v:.3f}" if isinstance(v, (int, float)) and v is not None else "-"


# 输出
if tabulate:
    # 高亮最大值
    # 预计算每列最大值
    col_max = {}
    for j, e in enumerate(eval_types, start=1):
        col_vals = [r[j] for r in table if isinstance(r[j], (int, float))]
        col_max[e] = max(col_vals) if col_vals else None
    pretty_table = []
    for r in table:
        new_r = [r[0]]
        for j, e in enumerate(eval_types, start=1):
            v = r[j]
            fv = format_val(v)
            if col_max[e] is not None and v == col_max[e]:
                fv = f"{Fore.GREEN}{Style.BRIGHT}{fv}{Style.RESET_ALL}"
            new_r.append(fv)
        pretty_table.append(new_r)
    print(tabulate(pretty_table, headers=headers,
          tablefmt="grid", stralign="center", numalign="center"))
else:
    # 简单对齐
    col_widths = [max(len(h), 8) for h in headers]
    def pad(s, w): return str(s).ljust(w)
    header_line = " | ".join(pad(h, w) for h, w in zip(headers, col_widths))
    print(header_line)
    print("-" * len(header_line))
    for r in table:
        print(" | ".join(pad(format_val(v) if i else pad(r[0], col_widths[0]).strip(),
                             col_widths[i]) for i, v in enumerate(r)))

if __name__ == "__main__":
    pass